---
title: "CLASIFICACIÓN DE ENFERMEDADES DE CACAO"
author: "Diego Guanotasig - Jorge Delgado"
date: "12-09-25"
format: html
jupyter: python3
---

# TRABAJO FINAL
### Librerias

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
BASE_DIR = "G:\Mi unidad\Colab Notebooks\AProfundo\clasificacion_dataset/"
IMG_SIZE = (224, 224)
BATCH = 32

In [ ]:
train_gen = ImageDataGenerator(rescale=1/255)
val_gen   = ImageDataGenerator(rescale=1/255)

train_ds = train_gen.flow_from_directory(
    BASE_DIR+"train",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical"
)

val_ds = val_gen.flow_from_directory(
    BASE_DIR+"val",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical"
)

num_classes = len(train_ds.class_indices)

In [ ]:
import matplotlib.pyplot as plt

# Mostrar 5 imágenes del dataset de entrenamiento
def mostrar_5_iniciales(dataset):
    images, labels = next(dataset)  # primer batch
    plt.figure(figsize=(12, 8))

    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i])
        plt.axis("off")

    plt.suptitle("Primeras 5 imágenes del dataset (train)")
    plt.show()

mostrar_5_iniciales(train_ds)

# EfficientNetB0

In [ ]:
efficient_model = EfficientNetB0(
include_top=False,
input_shape=IMG_SIZE+(3,),
weights="imagenet"
)

efficient_model.trainable = False

model_e = models.Sequential([
efficient_model,
layers.GlobalAveragePooling2D(),
layers.Dense(256, activation="relu"),
layers.Dense(num_classes, activation="softmax")
])

model_e.compile(
optimizer="adam",
loss="categorical_crossentropy",
metrics=["accuracy"]
)

history_e = model_e.fit(train_ds, validation_data=val_ds, epochs=10)

# MobileNetV2

In [ ]:
mobile = MobileNetV2(
include_top=False,
input_shape=IMG_SIZE+(3,),
weights="imagenet"
)
mobile.trainable = False

model_m = models.Sequential([
mobile,
layers.GlobalAveragePooling2D(),
layers.Dense(256, activation="relu"),
layers.Dense(num_classes, activation="softmax")
])

model_m.compile(
optimizer="adam",
loss="categorical_crossentropy",
metrics=["accuracy"]
)

history_m = model_m.fit(train_ds, validation_data=val_ds, epochs=10)

# Gráficas de Entrenamiento

In [ ]:
plt.plot(history_e.history['accuracy'], label="EfficientNet Acc")
plt.plot(history_m.history['accuracy'], label="MobileNet Acc")
plt.legend(); plt.title("Comparación Accuracy"); plt.show()

# Predicción desde imagenes celular

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

def predict(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, 0)
    
    pred = model.predict(img_array)[0]

    classes = list(train_ds.class_indices.keys())
    result = classes[np.argmax(pred)]

    # 🔹 Mostrar la imagen y la predicción
    plt.imshow(image.load_img(img_path))
    plt.title(f"Predicción: {result}")
    plt.axis("off")
    plt.show()

    return result, pred


img_path = r"G:\Mi unidad\Colab Notebooks\AProfundo\clasificacion_dataset\test\monilia\Monilia101.jpg"
print(predict(img_path, model_m))